In [14]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load
file_to_load = 'Resources/purchase_data.csv'

# Read Purchasing File and store into Pandas data frame
purchase_data_df = pd.read_csv(file_to_load)

## 1.) Player Count
* Total number of players

In [2]:
# Use unique() function to get array of unique players, and wrap in length to count them
total_unique_players = len(purchase_data_df['SN'].unique())
total_unique_players_df = pd.DataFrame({"Total Players":[total_unique_players]})
total_unique_players_df

,Total Players
0,576


## 2.) Purchasing Analysis
Calculate the following and load into summary dataframe:
* Number of unique items
* Average purchase price
* Total number of purchases
* Total revenue

In [3]:
# Use unique() function to calculate number of unique items
total_unique_items = len(purchase_data_df['Item ID'].unique())

# Average price 
average_price = purchase_data_df['Price'].mean()

# Total number of purchases
total_number_purchases = len(purchase_data_df.index)

# Total revenue
total_revenue = purchase_data_df['Price'].sum()

# Create dataframe
purchasing_analysis_summary_df = pd.DataFrame({'Number of Unique Items':[total_unique_items],
                                               'Average Price':[average_price],
                                               'Number of Purchases':[total_number_purchases],
                                               'Total Revenue':[total_revenue]})

# Format Average Price and Total Revenue Values
purchasing_analysis_summary_df['Average Price'] = purchasing_analysis_summary_df['Average Price'].map('${:,.2f}'.format)
purchasing_analysis_summary_df['Total Revenue'] = purchasing_analysis_summary_df['Total Revenue'].map('${:,.2f}'.format)

purchasing_analysis_summary_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## 3.) Gender Demographics
* Percentage and count of male players
* Percentage and count of female players
* Percentage and count of other/nondislosed

In [4]:
# Create new dataframe grouped by gender
# NOTE: approach came from https://stackoverflow.com/questions/30530663/how-to-select-distinct-across-multiple-data-frame-columns-in-pandas

# Create new data frame containing SN and gender 
gender_df = purchase_data_df[['SN','Gender']]
# Then use .drop_duplicates() function and load into new dataframe
unique_players_gender_df = gender_df.drop_duplicates()

# Group by gender
unique_players_gender_grouped_df = unique_players_gender_df.groupby(['Gender']).count()
# rename 'SN' column
unique_players_gender_grouped_df.columns = ['Total Count']

# Calculate % of Total
unique_players_gender_grouped_df['Percentage of Players'] = unique_players_gender_grouped_df['Total Count'] / unique_players_gender_grouped_df['Total Count'].sum()

# Reformat % of Total
unique_players_gender_grouped_df['Percentage of Players'] = unique_players_gender_grouped_df['Percentage of Players'].map('{:.2%}'.format)

#Sort descending 
unique_players_gender_grouped_df.sort_values(['Total Count'],ascending=False)




,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


## 4.) Purchasing Analysis (Gender)
* The below each broken out by gender:
    * Purchase count
    * Average purchase price
    * Total purchase value
    * Average purchase total per person by gender

In [5]:
# Create dataframe with just gender, purchase ID, and price
gender_purchase_df = purchase_data_df[['Gender','Purchase ID','Price']]

# Build separate dataframes to calculate each component of final summary table
# Count Purchase ID by gender
purchase_count_df = gender_purchase_df.groupby('Gender')[['Purchase ID']].count()
# Rename column to Purchase count, as it will be starting point for final dataframe
purchase_count_df.columns = ['Purchase Count']

# Average purchase price by gender
avg_purchase_price_df = gender_purchase_df.groupby('Gender')[['Price']].mean()

# Total purchase value by gender
totaL_purchase_value_df = gender_purchase_df.groupby('Gender')[['Price']].sum()

# Average purchase price per person
# Utilize sum from unique_players_gender_grouped_df from step 3 above
avg_price_per_person_df = gender_purchase_df.groupby('Gender')[['Price']].sum()
avg_price_per_person_df['Player Count'] = unique_players_gender_grouped_df['Total Count']
avg_price_per_person_df['Avg Total Purchase per Person'] = avg_price_per_person_df['Price'] / avg_price_per_person_df['Player Count']

# Build final dataframe based off of calculated values
gender_purchase_final_df = purchase_count_df
gender_purchase_final_df['Average Purchase Price'] = avg_purchase_price_df['Price']
gender_purchase_final_df['Total Purchase Value'] = totaL_purchase_value_df['Price']
gender_purchase_final_df['Avg Total Purchase per Person'] = avg_price_per_person_df['Avg Total Purchase per Person']

# Format values
gender_purchase_final_df['Average Purchase Price'] = gender_purchase_final_df['Average Purchase Price'].map('${:,.2f}'.format)
gender_purchase_final_df['Total Purchase Value'] = gender_purchase_final_df['Total Purchase Value'].map('${:,.2f}'.format)
gender_purchase_final_df['Avg Total Purchase per Person'] = gender_purchase_final_df['Avg Total Purchase per Person'].map('${:,.2f}'.format)

gender_purchase_final_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## 5.) Age Demographics
* Establish bins for ages (5 year bins)
* Calculate count and percentages of players by age group

In [27]:
# Pull SN, age for starting point to create summary dataframe
age_demographics_df = purchase_data_df[['SN','Age']]

# Add in age bands
# Create bin values to group records by age
bins = [0,9,14,19,24,29,34,39,np.inf]
# Create names for bins
group_names = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']
# Use cut function to add Age Band column
age_demographics_df['Age Band'] = pd.cut(age_demographics_df['Age'],bins,labels=group_names,include_lowest=True)

# Get distinct list of players and their ages
# NOTE: approach came from https://stackoverflow.com/questions/30530663/how-to-select-distinct-across-multiple-data-frame-columns-in-pandas
# Don't need to bring in age into this dataframe, just age band
unique_age_demographics_df = age_demographics_df[['SN','Age Band']].drop_duplicates() 

# Group by age band
unique_age_demographics_grouped_df = unique_age_demographics_df.groupby(['Age Band']).count()

# rename 'SN' column
unique_age_demographics_grouped_df.columns = ['Total Count']

# Calculate % of Total
unique_age_demographics_grouped_df['Percentage of Players'] = unique_age_demographics_grouped_df['Total Count'] / unique_age_demographics_grouped_df['Total Count'].sum()

# Reformat % of Total
unique_age_demographics_grouped_df['Percentage of Players'] = unique_age_demographics_grouped_df['Percentage of Players'].map('{:.2%}'.format)

unique_age_demographics_grouped_df





<ipython-input-27-6b0e9f5a67b3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_demographics_df['Age Band'] = pd.cut(age_demographics_df['Age'],bins,labels=group_names,include_lowest=True)


,Total Count,Percentage of Players
Age Band,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## 6.) Purchasing Analysis (Age)
* The below each broken out by age band defined in step 5 above:
    * Purchase count
    * Average purchase price
    * Total purchase value
    * Average purchase total per person by gender

In [46]:
# Create dataframe with just age, purchase ID, and price
age_purchase_df = purchase_data_df[['Age','Purchase ID','Price']]

# Create age band column
# Create bin values to group records by age
bins = [0,9,14,19,24,29,34,39,np.inf]
# Create names for bins
group_names = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']
# Use cut function to add Age Band column
age_purchase_df['Age Band'] = pd.cut(age_purchase_df['Age'],bins,labels=group_names,include_lowest=True)

# Remove age column since it is no longer necessary for this analysis now that age band is created
# NOTE: Use approach from https://stackoverflow.com/questions/13411544/delete-column-from-pandas-dataframe
age_purchase_df = age_purchase_df.drop('Age',1)

# Now, to create summary table, use similar approach from step 4 above
# Build separate dataframes to calculate each component of final summary table

# Count Purchase ID by age band
purchase_count_df = age_purchase_df.groupby('Age Band')[['Purchase ID']].count()
# Rename column to Purchase count, as it will be starting point for final dataframe
purchase_count_df.columns = ['Purchase Count']

# Average purchase price by age band
avg_purchase_price_df = age_purchase_df.groupby('Age Band')[['Price']].mean()

# Total purchase value by age band
totaL_purchase_value_df = age_purchase_df.groupby('Age Band')[['Price']].sum()

# Average purchase price per person
# Utilize sum from unique_age_demographics_grouped_df from step 5 above
avg_price_per_person_df = age_purchase_df.groupby('Age Band')[['Price']].sum()
avg_price_per_person_df['Player Count'] = unique_age_demographics_grouped_df['Total Count']
avg_price_per_person_df['Avg Total Purchase per Person'] = avg_price_per_person_df['Price'] / avg_price_per_person_df['Player Count']

# Build final dataframe based off of calculated values
age_purchase_final_df = purchase_count_df
age_purchase_final_df['Average Purchase Price'] = avg_purchase_price_df['Price']
age_purchase_final_df['Total Purchase Value'] = totaL_purchase_value_df['Price']
age_purchase_final_df['Avg Total Purchase per Person'] = avg_price_per_person_df['Avg Total Purchase per Person']

# Format values
age_purchase_final_df['Average Purchase Price'] = age_purchase_final_df['Average Purchase Price'].map('${:,.2f}'.format)
age_purchase_final_df['Total Purchase Value'] = age_purchase_final_df['Total Purchase Value'].map('${:,.2f}'.format)
age_purchase_final_df['Avg Total Purchase per Person'] = age_purchase_final_df['Avg Total Purchase per Person'].map('${:,.2f}'.format)

age_purchase_final_df



<ipython-input-46-2cc61dc7bd1e>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_purchase_df['Age Band'] = pd.cut(age_purchase_df['Age'],bins,labels=group_names,include_lowest=True)


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Band,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19
